# 4c Improving the training loop


In [ ]:
network = nn.Sequential(collections.OrderedDict([
            ("hidden", nn.Linear(28*28,256)),
            ("sigmoid", nn.Sigmoid()),
            ("output", nn.Linear(256,10)),
            ("softmax", nn.Softmax(dim=1))
          ]))

In [ ]:
class MnistDataSet:
    def __init__(self, train=True):
        subset = "training" if train else "test"
        self.x, self.y = torch.load(data.DATA_PATH / f"MNIST/processed/{subset}.pt")
    
    def __getitem__(self, key) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.x[key], self.y[key]
    
    def __len__(self):
        return len(self.x)
    
class DigitFlipper:
    def __init__(self, digits_to_flip: List[int], prob=0.5):
        self.digits_to_flip, self.prob = digits_to_flip, prob
        
    def __call__(self, x, y):
        for i, digit in enumerate(y.numpy()):
            if digit in self.digits_to_flip and random.random() <= self.prob:
                x[i] = x[i].flip(-1)
        return x, y
    
class MnistDataLoader:
    def __init__(self, dataset, batch_size, shuffle, transform=None):
        self.dataset, self.batch_size, self.shuffle, self.transform = dataset, batch_size, shuffle, transform
    
    def __iter__(self):
        self.idx = list(range(len(self.dataset)))
        if self.shuffle:
            random.shuffle(self.idx)
        return self
    
    def __next__(self):
        if self.idx:
            batch, self.idx = self.idx[:self.batch_size], self.idx[self.batch_size:]
            x, y = self.dataset[batch]
            if self.transform:
                return self.transform(x, y)
            return x, y
        
        raise StopIteration()

def cross_entropy(target, prediction):
    return - target[:, prediction.argmax()].log().mean()

n_epochs = 10
train_dl = MnistDataLoader(MnistDataSet(train=True), 1024, True)
test_dl = MnistDataLoader(MnistDataSet(train=False), 1024, False)

for epoch in range(n_epochs):
    print(f"Epoch #{epoch+1}")
    
    for x, y in train_dl:
        